Otimização da Função de Ackley

Grupo: Anderson Carneiro da Silva (acs6@cin.ufpe.br) e Lucas Thierry Chaves Muniz (ltcm@cin.ufpe.br)

In [112]:
from numpy import pi
from numpy import cos
from numpy import exp
from numpy import sqrt

Função Fitness

In [113]:
def ackley (vector, c1=20, c2=0.2, c3=2*pi):
    d = len(vector)
    sqrt_sum=0
    cos_sum=0
    for elem in vector:
        sqrt_sum += elem**2
        cos_sum += cos(c3*elem)
    
    sqrt_term = -c1 * exp(-c2*(sqrt(sqrt_sum/d)))
    cos_term = -exp(cos_sum/d)

    return sqrt_term + cos_term + c1 + 1

In [114]:
print(ackley([0, 0, 0, 0]))

-1.7182818284590446
